<a href="https://colab.research.google.com/github/rmgithub123/be_class/blob/main/DecisionTree_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [2]:
# Download necessary NLTK data
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/ds_csv_file/emails.csv')

In [4]:
data

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Email 861,3,0,1,1,0,1,11,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
861,Email 862,0,1,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
862,Email 863,51,48,112,16,20,12,442,8,54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
863,Email 864,1,6,8,1,1,0,41,3,2,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data.describe()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
count,865.000000,865.000000,865.000000,865.000000,865.000000,865.000000,865.000000,865.000000,865.000000,865.000000,...,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000
mean,7.289017,6.771098,5.916763,3.159538,3.492486,2.707514,61.804624,2.828902,2.346821,11.363006,...,0.016204,0.008102,0.010417,0.096065,0.008102,0.001157,0.012731,0.981481,0.006944,0.283565
std,13.420106,10.811874,14.230168,6.501779,5.100404,6.342738,110.616852,4.847761,7.170376,20.264227,...,0.197825,0.122466,0.112417,0.499161,0.122466,0.034021,0.139773,4.297686,0.083092,0.450989
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,15.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,3.000000,2.000000,1.000000,2.000000,1.000000,30.000000,1.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,8.000000,5.000000,3.000000,4.000000,2.000000,62.000000,3.000000,2.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,210.000000,132.000000,188.000000,83.000000,39.000000,77.000000,1898.000000,42.000000,92.000000,223.000000,...,4.000000,3.000000,2.000000,6.000000,3.000000,1.000000,3.000000,114.000000,1.000000,1.000000


In [6]:
# Compute basic statistics
summary_statistics = data.describe()
print("Basic Statistics:\n", summary_statistics)

Basic Statistics:
               the          to         ect         and         for          of  \
count  865.000000  865.000000  865.000000  865.000000  865.000000  865.000000   
mean     7.289017    6.771098    5.916763    3.159538    3.492486    2.707514   
std     13.420106   10.811874   14.230168    6.501779    5.100404    6.342738   
min      0.000000    0.000000    1.000000    0.000000    0.000000    0.000000   
25%      1.000000    1.000000    1.000000    0.000000    1.000000    0.000000   
50%      3.000000    3.000000    2.000000    1.000000    2.000000    1.000000   
75%      8.000000    8.000000    5.000000    3.000000    4.000000    2.000000   
max    210.000000  132.000000  188.000000   83.000000   39.000000   77.000000   

                 a         you         hou          in  ...    connevey  \
count   865.000000  865.000000  865.000000  865.000000  ...  864.000000   
mean     61.804624    2.828902    2.346821   11.363006  ...    0.016204   
std     110.616852    4.84

In [10]:
data['Prediction'].value_counts()

Prediction
0.0    619
1.0    245
Name: count, dtype: int64

In [14]:
data.isnull().sum()

Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      1
allowing      1
ff            1
dry           1
Prediction    1
Length: 3002, dtype: int64

In [11]:
x=data.iloc[:,:-1]
y=data['Prediction']

In [15]:
data.dtypes

Email No.      object
the             int64
to              int64
ect             int64
and             int64
               ...   
military      float64
allowing      float64
ff            float64
dry           float64
Prediction    float64
Length: 3002, dtype: object

In [17]:
data = data.fillna(0)

In [18]:
data.isnull().sum()

Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      0
allowing      0
ff            0
dry           0
Prediction    0
Length: 3002, dtype: int64

In [26]:
x=data.iloc[:,1:-1]
y=data['Prediction']

In [27]:
x

,the,to,ect,and,for,of,a,you,hou,in,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,0,0,1,0,0,0,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,13,24,6,6,2,102,1,27,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,1,0,0,0,8,0,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,5,22,0,5,1,51,2,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,6,17,1,5,2,57,0,9,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,3,0,1,1,0,1,11,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
861,0,1,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
862,51,48,112,16,20,12,442,8,54,63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
863,1,6,8,1,1,0,41,3,2,6,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
860    1.0
861    1.0
862    0.0
863    0.0
864    0.0
Name: Prediction, Length: 865, dtype: float64

In [29]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [30]:
# Build the Decision Tree model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [31]:
# Predict on the test dataset
y_pred = model.predict(X_test)

In [32]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [33]:
y_pred

array([1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 1.])

In [34]:
# Print evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Accuracy: 0.8959537572254336
Precision: 0.8125
Recall: 0.8125
F1-Score: 0.8125


Precision Score

Precision is the ratio of correctly predicted positive observations to the total predicted positives. It answers the question: "Of all the emails predicted as spam, how many are actually spam?"

Precision=TP/FP+TP​

    TP (True Positives): The number of correctly predicted positive examples (spam emails correctly identified as spam).
    FP (False Positives): The number of incorrectly predicted positive examples (non-spam emails incorrectly identified as spam).

Recall Score

Recall (also known as Sensitivity or True Positive Rate) is the ratio of correctly predicted positive observations to all the observations in the actual class. It answers the question: "Of all the actual spam emails, how many were correctly identified as spam?"

Recall=TP/TP+FN

    TP (True Positives): The number of correctly predicted positive examples (spam emails correctly identified as spam).
    FN (False Negatives): The number of actual positive examples that were incorrectly predicted as negative (spam emails incorrectly identified as non-spam).

F1 Score

The F1 score is the harmonic mean of precision and recall. It is a single metric that combines both precision and recall, and is particularly useful when you need to balance both concerns in your model. It provides a better measure than accuracy in cases where the class distribution is imbalanced.

F1 Score=2⋅(Precision.Recall / Precision+Recall​)

Why These Metrics Are Important



*   Precision is crucial when the cost of false positives is high. For example, in email spam detection, marking a legitimate email as spam could cause important emails to be missed.

*   Recall is important when the cost of false negatives is high. In spam detection, missing a spam email might be less critical, but in medical diagnostics, missing a positive case (e.g., a disease) can be very serious.

*   F1 Score provides a balance between precision and recall and is a good metric to use when you need to consider both false positives and false negatives.






Example Calculation

Assume we have the following confusion matrix for a binary classification problem:
	Predicted: Spam	Predicted: Not Spam
Actual: Spam	TP = 70	FN = 30
Actual: Not Spam	FP = 10	TN = 90

    Precision: 7070+10=7080=0.87570+1070​=8070​=0.875 or 87.5%
    Recall: 7070+30=70100=0.770+3070​=10070​=0.7 or 70%
    F1 Score: 2⋅0.875⋅0.70.875+0.7=2⋅0.61251.575≈0.7782⋅0.875+0.70.875⋅0.7​=2⋅1.5750.6125​≈0.778 or 77.8%

These metrics help to understand the performance of the classification model in different aspects and make informed decisions on improving it.